In [1]:
import sys, os

# Path of current working directory (where notebook is running)
cwd = os.getcwd()
sys.path.append(os.path.join(cwd, "backdoors101"))

import os
os.environ.pop("PYTORCH_CUDA_ALLOC_CONF", None)

'max_split_size_mb:128,expandable_segments:True'

In [ ]:
# eval_backdoor.py
import torch
from torch.utils.data import DataLoader
from tasks.task import Task   # repo Task loader (adjust import as needed)
from attack import make_synthesizer  # or however synths are constructed
from datasets import make_test_dataset  # pseudo-import - adapt to repo


ModuleNotFoundError: No module named 'synths'

In [ ]:

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_PATH = "saved_models/your_run/final_model.pt"
PARAMS_PATH = "configs/mnist_params.yaml"  # used to construct same Task/synth


In [ ]:

# 1) load model + task (so we know preprocessing)
task = Task.from_params(PARAMS_PATH, train=False)  # or use Helper to build Task
model = task.model
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.to(DEVICE)
model.eval()

# 2) build test dataloader (clean)
test_ds = task.make_test_dataset()  # repo-specific helper
dl = DataLoader(test_ds, batch_size=256, shuffle=False, num_workers=4)

# 3) get synthesizer (same one used in train)
synth = make_synthesizer(PARAMS_PATH)  # construct with same params

total = 0
clean_correct = 0
asr_count = 0
asr_total = 0

target_label = task.params.attack.target_label  # the attacker target

with torch.no_grad():
    for x, y in dl:
        x = x.to(DEVICE)
        y = y.to(DEVICE)

        # clean
        logits = model(x)
        preds = logits.argmax(dim=1)
        clean_correct += (preds == y).sum().item()

        # triggered version (synthesizer takes a batch and returns backdoored x)
        x_t = synth.apply_trigger_batch(x)   # repo-specific API
        logits_t = model(x_t)
        preds_t = logits_t.argmax(dim=1)
        # count how many triggered examples mapped to target_label
        asr_count += (preds_t == target_label).sum().item()
        asr_total += x_t.shape[0]

        total += x.shape[0]

clean_acc = clean_correct / total
asr = asr_count / asr_total
print(f"Clean accuracy: {clean_acc:.4f} | ASR (target {target_label}): {asr:.4f}")


In [ ]:
# load params used during training
with open("./config_loss.yml") as f:
    params = yaml.safe_load(f)
params['name'] = 'mnist_eval'
params['log'] = False
params

In [ ]:
# instantiate helper (it builds task/model)
helper = Helper(params)
task = helper.task
model = task.model
device = helper.params.device
model.to(device)
model.eval()

In [ ]:
# load a saved model if training saved one
# helper.load_model('paper_presentation/saved_models/model_MNIST_Sep.30_22.41.29_mnist_loss_test_run/model_last.pt.tar')   # adjust path if needed
# 

In [ ]:
# Clean accuracy
correct = total = 0
test_loader = DataLoader(task.test_dataset, batch_size=helper.params.test_batch_size)
for X, y in test_loader:
    X = X.to(device); y = y.to(device)
    with torch.no_grad():
        logits = model(X)
        preds = logits.argmax(dim=1)
    correct += (preds == y).sum().item()
    total += y.size(0)
print("Clean acc:", correct / total)

In [ ]:
# ASR: poison the same test samples with the same synthesizer used in training
backdoor_label = helper.params.backdoor_label
correct = total = 0
synth = helper.attack.synthesizer  # repo typically exposes attack.synthesizer
for X, y in test_loader:
    # synth.synthesize_batch is pseudocode — replace with actual synthesizer API
    Xp = synth.synthesize_batch(X, target_label=backdoor_label)  # adapt to repo method
    Xp = Xp.to(device)
    with torch.no_grad():
        logits = model(Xp)
        preds = logits.argmax(dim=1)
    correct += (preds == backdoor_label).sum().item()
    total += preds.size(0)
print("ASR:", correct / total)
